# Configuration for Contextual RAG with Amazon Bedrock

This notebook sets up the necessary AWS configurations for a Contextual RAG (Retrieval-Augmented Generation) system using Amazon Bedrock and OpenSearch. Follow the steps sequentially to configure your environment properly.

## Import Required Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

# Install required packages
%pip install -r requirements.txt

## 1. AWS Configuration
This section sets up the AWS credentials and region for subsequent operations.

In [ ]:
import os
import boto3
import json
from sagemaker import get_execution_role
import sys
import requests
from pprint import pprint

# Get AWS role and session
role = get_execution_role()
session = boto3.Session()

## 2. Bedrock Configuration
Configure Amazon Bedrock service with appropriate model IDs.

In [ ]:
from utils.bedrock import BedrockClient

# List available foundation models
print("\n== Available Foundation Models ==")
all_models = BedrockClient.get_list_fm_models()
pprint(all_models)

In [ ]:
# Set Bedrock model IDs
# Claude 3.5 Sonnet for text generation
os.environ["BEDROCK_MODEL_ID"] = "anthropic.claude-3-5-sonnet-20241022-v2:0"
# Titan Embeddings for vector generation
os.environ["EMBED_MODEL_ID"] = "amazon.titan-embed-text-v2:0"
os.environ["BEDROCK_RETRIES"] = "10"

print(f"\nBedrock Models configured:")
print(f"- Text Generation: {os.environ['BEDROCK_MODEL_ID']}")
print(f"- Embeddings: {os.environ['EMBED_MODEL_ID']}")

## 3. Opensearch Configuration
Set up the OpenSearch service for vector storage and retrieval.

### 3-1. Index Configuration

In [ ]:
from utils.ssm import parameter_store

# Create parameter store for configurations
region = boto3.Session().region_name
pm = parameter_store(region)

# Define index name for OpenSearch
index_name = "default_doc_index" 

# Store the index name in parameter store
pm.put_params(
    key="opensearch_index_name",
    value=f'{index_name}',
    overwrite=True,
    enc=False
)
print(f"Index name '{index_name}' stored in parameter store")

### 3-2. Define Index Schema
The schema includes support for Korean text analysis using Nori plugin and vector search capabilities.

In [ ]:
# Define vector dimension for embeddings
dimension = 1024

# Define the index schema with Nori analyzer for Korean text
index_body = {
    'settings': {
        'analysis': {
            'analyzer': {
                'my_analyzer': {
                    'char_filter': ['html_strip'],
                    'tokenizer': 'nori',
                    'filter': [
                        'my_nori_part_of_speech'
                    ],
                    'type': 'custom'
                }
            },
            'tokenizer': {
                'nori': {
                    'decompound_mode': 'mixed',
                    'discard_punctuation': 'true',
                    'type': 'nori_tokenizer'
                }
            },
            "filter": {
                "my_nori_part_of_speech": {
                    "type": "nori_part_of_speech",
                    "stoptags": [
                        "J", "XSV", "E", "IC", "MAJ", "NNB",
                        "SP", "SSC", "SSO",
                        "SC", "SE", "XSN", "XSV",
                        "UNA", "NA", "VCP", "VSV",
                        "VX"
                    ]
                }
            }
        },
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Cosine similarity for vector search
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                    'source': {'type': 'keyword'},
                    'page_number': {'type': 'long'},
                    'category': {'type': 'text'},
                    'file_directory': {'type': 'text'},
                    'last_modified': {'type': 'text'},
                    'type': {'type': 'keyword'},
                    'image_base64': {'type': 'text'},
                    'origin_image': {'type': 'text'},
                    'origin_table': {'type': 'text'},
                }
            },
            'text': {
                'analyzer': 'my_analyzer',
                'search_analyzer': 'my_analyzer',
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': dimension
            }
        }
    }
}

### 3-3. Get Opensearch Domain Information

In [ ]:
# Retrieve OpenSearch domain endpoint from parameter store
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

# Get authentication credentials from AWS Secrets Manager
secrets_manager = boto3.client('secretsmanager')
response = secrets_manager.get_secret_value(
    SecretId='opensearch_user_password'
)

secrets_string = response.get('SecretString')
secrets_dict = eval(secrets_string)

opensearch_user_id = secrets_dict['es.net.http.auth.user']
opensearch_user_password = secrets_dict['pwkey']

http_auth = (opensearch_user_id, opensearch_user_password)

# Parse OpenSearch domain endpoint
result = pm.parse_opensearch_endpoint(opensearch_domain_endpoint)
prefix, domain_name = result

# Set OpenSearch environment variables
os.environ["OPENSEARCH_PREFIX"] = ""
os.environ["OPENSEARCH_DOMAIN_NAME"] = "contextual-rag-domain"
os.environ["OPENSEARCH_DOCUMENT_NAME"] = ""
os.environ["OPENSEARCH_USER"] = opensearch_user_id
os.environ["OPENSEARCH_PASSWORD"] = opensearch_user_password

print(f"OpenSearch Domain: {domain_name}")
print(f"OpenSearch Prefix: {prefix}")
print(f"OpenSearch User: {opensearch_user_id}")

### 3-4. Create Opensearch Client

In [ ]:
from utils.opensearch import opensearch_utils

# Create OpenSearch client
aws_region = os.environ.get("AWS_DEFAULT_REGION")
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

# Check if index exists
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)
print(f"Index '{index_name}' exists: {index_exists}")

### 3-5. Check OpenSearch Plugins
Before creating the index, let's verify that the necessary plugins (especially Nori for Korean text) are installed.

In [ ]:
# Check installed plugins in OpenSearch
plugins = os_client.cat.plugins(format="json")
print("Installed Plugins:")
for plugin in plugins:
    print(plugin['component'])

# Check if Nori plugin exists
nori_installed = any("analysis-nori" in plugin['component'] for plugin in plugins)
if not nori_installed:
    print("\n⚠️ WARNING: Nori plugin for Korean text analysis is not installed!")
    print("To install Nori plugin, follow the instructions at:")
    print("https://aws.amazon.com/blogs/tech/amazon-opensearch-service-korean-nori-plugin-for-analysis/")
    print("You'll need to use a compatible OpenSearch version and add the plugin through the console.")
    print("Without this plugin, Korean text analysis will not work properly.\n")
else:
    print("\n✅ Nori plugin is installed. Korean text analysis is supported.")

### 3-6. Create Index

In [ ]:
# Check again if index exists
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

# Delete index if it already exists
if index_exists:
    print(f"Deleting existing index: {index_name}")
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

# Create new index
try:
    print(f"Creating index: {index_name}")
    opensearch_utils.create_index(os_client, index_name, index_body)
    index_info = os_client.indices.get(index=index_name)
    print("Index created successfully!")
    print("\nIndex Configuration:")
    pprint(index_info)
except Exception as e:
    print(f"Error creating index: {str(e)}")
    
    # If error is related to Nori plugin, provide additional guidance
    if "nori_tokenizer" in str(e):
        print("\nError appears to be related to the Nori tokenizer.")
        print("Please ensure the Nori plugin is installed in your OpenSearch domain.")
        print("Follow the instructions at: https://aws.amazon.com/blogs/tech/amazon-opensearch-service-korean-nori-plugin-for-analysis/")

## 4. RAG Application Configuration
Additional configuration for the RAG application components.

In [ ]:
# Reranker Configuration
os.environ["RERANKER_AWS_REGION"] = "us-west-2"
os.environ["RERANKER_AWS_PROFILE"] = ""
os.environ["RERANKER_MODEL_ID"] = "amazon.rerank-v1:0"

# Rank Fusion Configuration
os.environ["RERANK_TOP_K"] = "20"
os.environ["HYBRID_SCORE_FILTER"] = "40"
os.environ["FINAL_RERANKED_RESULTS"] = "20"
os.environ["KNN_WEIGHT"] = "0.6"

# Application Configuration
os.environ["RATE_LIMIT_DELAY"] = "60"  # API 요청 간 지연 시간(초) (기본값: 60)

## 5. Save Configuration to .env File
Create a .env file to easily share configurations between notebooks.

In [ ]:
# Create .env file with all configurations
env_vars = {
    # AWS Configuration
    "AWS_REGION": os.environ.get("AWS_DEFAULT_REGION", "us-west-2"),
    "AWS_PROFILE": os.environ.get("AWS_PROFILE", "default"),
    
    # Bedrock Configuration
    "BEDROCK_MODEL_ID": os.environ.get("BEDROCK_MODEL_ID", ""),
    "EMBED_MODEL_ID": os.environ.get("EMBED_MODEL_ID", ""),
    "BEDROCK_RETRIES": os.environ.get("BEDROCK_RETRIES", "10"),
    
    # Model Configuration
    "MAX_TOKENS": os.environ.get("MAX_TOKENS", "4096"),
    "TEMPERATURE": os.environ.get("TEMPERATURE", "0"),
    "TOP_P": os.environ.get("TOP_P", "0.7"),
    
    # OpenSearch Configuration
    "OPENSEARCH_PREFIX": os.environ.get("OPENSEARCH_PREFIX", ""),
    "OPENSEARCH_DOMAIN_NAME": os.environ.get("OPENSEARCH_DOMAIN_NAME", ""),
    "OPENSEARCH_DOCUMENT_NAME": os.environ.get("OPENSEARCH_DOCUMENT_NAME", ""),
    "OPENSEARCH_USER": os.environ.get("OPENSEARCH_USER", ""),
    "OPENSEARCH_PASSWORD": os.environ.get("OPENSEARCH_PASSWORD", ""),
    
    # Reranker Configuration
    "RERANKER_MODEL_ID": os.environ.get("RERANKER_MODEL_ID", "amazon.rerank-v1:0"),
    "RERANKER_AWS_REGION": os.environ.get("RERANKER_AWS_REGION", "us-west-2"),
    "RERANKER_AWS_PROFILE": os.environ.get("RERANKER_AWS_PROFILE", ""),
    
    # Rank Fusion Configuration
    "RERANK_TOP_K": os.environ.get("RERANK_TOP_K", "20"),
    "HYBRID_SCORE_FILTER": os.environ.get("HYBRID_SCORE_FILTER", "40"),
    "FINAL_RERANKED_RESULTS": os.environ.get("FINAL_RERANKED_RESULTS", "20"),
    "KNN_WEIGHT": os.environ.get("KNN_WEIGHT", "0.6"),
    
    # Application Configuration
    "CHUNK_SIZE": os.environ.get("CHUNK_SIZE", "1000"),
    "RATE_LIMIT_DELAY": os.environ.get("RATE_LIMIT_DELAY", "60")
}

# Write .env file
with open('.env', 'w') as f:
    for key, value in env_vars.items():
        if value is not None and value != "":
            f.write(f"{key}={value}\n")

print(".env file created successfully!")
print("This file will be used by other notebooks to load the configuration.")

# Load configuration into config object
try:
    from config import Config
    config = Config.load()
    
    # Environment variables will be automatically loaded by the Config class
    # when instantiated via the load() method, so no need to manually update values
    
    print("\nConfiguration loaded successfully!")
    print("Current configuration:")
    print(f"AWS Config: {config.aws}")
    print(f"Bedrock Config: {config.bedrock}")
    print(f"Model Config: {config.model}")
    print(f"OpenSearch Config: {config.opensearch}")
    print(f"Reranker Config: {config.reranker}")
    print(f"Rank Fusion Config: {config.rank_fusion}")
    print(f"App Config: {config.app}")
    
except ImportError:
    print("\nWarning: Couldn't import Config. Configuration will be loaded from .env in subsequent notebooks.")

print("\n✅ Configuration complete! You can now proceed to the next notebook.")